# Utils notebook

This python notebook has util function for data analysis

1) To generate the overview picture of dataset
2) To generte diffs files for all patches

### Following is the code to generate all_patches_stats later saved at results/smartbugs/data_analysis/all_patches_stats.csv

In [21]:
tools = [
    'Elysium', 
    'sGuard', 
    'sGuardPlus',
    'SmartFix', 
    'SmartShield', 
    "SolGPT" ,
    'TIPS', 
]

In [22]:
Columns=['Patch','Original','Category','Tool','DIFF','COMP', 'Detected','Fixed','Consistent','functional_check','mitigates','patch_link', 'original_link', 'diff_link', 'exploit_link']

In [23]:
import os

#collecting contracts name
dataset='smartbugs/data_analysis/'
results_path='../results/'
contracts_path=os.path.join(results_path,dataset,'contracts.csv')
storage_path=os.path.join(results_path,dataset)

import pandas as pd
contracts_df = pd.read_csv(contracts_path)
result_df = pd.DataFrame([], columns=Columns)

In [24]:
vulnerabilities=contracts_df["Category"].unique().tolist()
contract_names=contracts_df["Name"].unique().tolist()

In [25]:
## Verify the columns
result_df

,Patch,Original,Category,Tool,DIFF,COMP,Detected,Fixed,Consistent,functional_check,mitigates,patch_link,original_link,diff_link,exploit_link


In [26]:
import re
def get_contract(input_string):
    parts = input_string.split('/')
    return parts[-2]  

def get_category(text):
    folders = os.path.normpath(text).split(os.sep)
    
    # Check each folder against the predefined category list
    for folder in folders:
        if folder in vulnerabilities:
            return folder
    return "unknown" 
def get_patch(input_string):
    parts = input_string.split('/')
    return parts[-1]

In [27]:
#test

def test_n_patches(result_df):
    assert len(result_df[result_df['Tool'] == 'Elysium'].values.tolist()) == 121
    assert len(result_df[result_df['Tool'] == 'SmartShield'].values.tolist()) == 135
    assert len(result_df[result_df['Tool'] == 'sGuard'].values.tolist()) == 109
    assert len(result_df[result_df['Tool'] == 'sGuardPlus'].values.tolist()) == 81
    assert len(result_df[result_df['Tool'] == 'SmartFix'].values.tolist()) == 86
    assert len(result_df[result_df['Tool'] == 'TIPS'].values.tolist()) == 242
    assert len(result_df[result_df['Tool'] == 'SolGPT'].values.tolist()) == 552
def test_dataset(result_df):
    assert len(result_df['Original'].unique()) == 141
    assert len(result_df['Category'].unique()) ==10
    assert len(result_df['Tool'].unique()) == 7

def test_diff_patches(result_df):
    assert len(result_df[(result_df['Tool'] == 'Elysium') & (result_df['DIFF'] == True)]) == 95
    assert len(result_df[(result_df['Tool'] == 'SmartShield') & (result_df['DIFF'] == True)]) == 135
    assert len(result_df[(result_df['Tool'] == 'sGuard') & (result_df['DIFF'] == True)]) == 62
    assert len(result_df[(result_df['Tool'] == 'sGuardPlus') & (result_df['DIFF'] == True)]) == 81
    assert len(result_df[(result_df['Tool'] == 'SmartFix') & (result_df['DIFF'] == True)]) == 86
    assert len(result_df[(result_df['Tool'] == 'TIPS') & (result_df['DIFF'] == True)]) == 242
    assert len(result_df[(result_df['Tool'] == 'SolGPT') & (result_df['DIFF'] == True)]) == 552
def test_compilable_patches(result_df):
    assert len(result_df[(result_df['Tool'] == 'Elysium') & (result_df['COMP'] == True)]) == 0  # Elysium does not have compilable data
    assert len(result_df[(result_df['Tool'] == 'SmartShield') & (result_df['COMP'] == True)]) == 0  # Smartshield does not have compilable data
    assert len(result_df[(result_df['Tool'] == 'sGuard') & (result_df['COMP'] == True)]) == 108
    assert len(result_df[(result_df['Tool'] == 'sGuardPlus') & (result_df['COMP'] == True)]) == 81
    assert len(result_df[(result_df['Tool'] == 'SmartFix') & (result_df['COMP'] == True)]) == 86
    assert len(result_df[(result_df['Tool'] == 'TIPS') & (result_df['COMP'] == True)]) == 234
    assert len(result_df[(result_df['Tool'] == 'SolGPT') & (result_df['COMP'] == True)]) == 527

def test_valid_patches(result_df):
    assert len(result_df[(result_df['Tool'] == 'Elysium') & (result_df['Valid Patches'] == True)]) == 101  # Elysium does not have compilable patches
    assert len(result_df[(result_df['Tool'] == 'SmartShield') & (result_df['Valid Patches'] == True)]) == 131  # Smartshield does not have compilable patches
    assert len(result_df[(result_df['Tool'] == 'sGuard') & (result_df['Valid Patches'] == True)]) == 61
    assert len(result_df[(result_df['Tool'] == 'sGuardPlus') & (result_df['Valid Patches'] == True)]) == 81
    assert len(result_df[(result_df['Tool'] == 'SmartFix') & (result_df['Valid Patches'] == True)]) == 86
    assert len(result_df[(result_df['Tool'] == 'TIPS') & (result_df['Valid Patches'] == True)]) == 234
    assert len(result_df[(result_df['Tool'] == 'SolGPT') & (result_df['Valid Patches'] == True)]) == 527




In [28]:
def read_diff(tools, result_df):
    for tool in tools:
        tool_diff_file='../results/smartbugs/'+tool+'/patches_diff.csv'
        df_diff=pd.read_csv(tool_diff_file, names=['Patch', 'Original','DIFF'], skiprows=1)
        if tool in ['SolGPT']:
            df_diff=pd.read_csv(tool_diff_file, names=['Patch', 'Original','cleaned','DIFF'], skiprows=1)
        values=df_diff.values.tolist()
        for line in values:
            category= get_category(line[0])
            patch = get_patch(line[0])
            original= get_contract(line[0])+ '.sol'
            diff= line[-1]
            patch_link= 'https://github.com/ASSERT-KTH/RepairComp/blob/main/results/smartbugs/'+tool+'/'+line[0]
            original_link='https://github.com/smartbugs/smartbugs-curated/tree/main/dataset/'+category+'/'+original
            diff_link= 'https://github.com/ASSERT-KTH/RepairComp/blob/main/results/smartbugs/'+tool+'/'+line[0] .replace('.sol', '.diff')
            if tool in ['Elysium', 'SmartShield']:
                patch=patch[:-7]+'.bin'
                diff_link="n/a"
            new_row = {'Patch': patch, 'Category': category, 'Original': original, 'Tool':tool, 'DIFF': diff,'patch_link':patch_link, 'original_link':original_link, 'diff_link': diff_link}
            result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)
    return result_df
                   


In [29]:
def read_comp(tools,result_df):
    for tool in tools:
        tool_comp_file='../results/smartbugs/'+tool+'/compilation_results_0.4.24.csv'
        df_comp=pd.read_csv(tool_comp_file, names=['Patch', 'COMP'], skiprows=1)
        values=df_comp.values.tolist()
        for line in values:
            category= get_category(line[0])
            patch = get_patch(line[0])
            original= get_contract(line[0])+ '.sol'
            comp= line[-1]
            if comp == 0:
                comp = True
            else :
                comp = False
            mask = (result_df['Patch'] == patch) & (result_df['Category'] == category) & (result_df['Tool'] == tool)
            if mask.any():
                result_df.loc[mask, 'COMP'] = comp
    return result_df
    

In [30]:
def read_d_f_m(tools,result_df):
    for tool in tools:
        tool_dfm_file='../results/smartbugs/'+tool+'/patches_evaluation.csv'
        df_dfm=pd.read_csv(tool_dfm_file, names=['Patch', 'Original','Detected', 'Fixed','Maintained'], skiprows=1)
        values=df_dfm.values.tolist()
        for line in values:
            category= get_category(line[1])
            patch = get_patch(line[0])
            original= get_contract(line[0])+ '.sol'
            D= line[-3]
            F=line[-2]
            mask = (result_df['Patch'] == patch) & (result_df['Category'] == category) & (result_df['Tool'] == tool)
            result_df.loc[mask, 'Detected'] = D
            result_df.loc[mask, 'Fixed']= F
            result_df.loc[mask, 'Consistent'] = True if D == F and D>0 else False
    return result_df

In [31]:
#collecting exploits
#from curl to https://github.com/ASSERT-KTH/sb-heists/blob/main/smartbugs-curated/0.4.x/contracts_w_exploits.csv
import requests
import json
from io import StringIO

# Get the raw GitHub content
url = "https://raw.githubusercontent.com/ASSERT-KTH/sb-heists/main/smartbugs-curated/0.4.x/contracts_w_exploits.csv"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Read the CSV content from the response
    csv_content = StringIO(response.text)
    
    # Load the file paths
    exploits = pd.read_csv(csv_content, header=None, names=['file_path'])
    
    # Split the file_path into category and name
    exploits[['category', 'name']] = exploits['file_path'].str.split('/', n=1, expand=True)
    
    # Display the first few rows to verify
    print(f"Total files: {len(exploits)}")
    
    # Get a count of files by category
    print("\nFiles per category:")
    print(exploits['category'].value_counts())
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

def mark_no_exploit_contracts(result_df, exploits_df):
    """
    Mark contracts with no associated exploit by setting functional_check, mitigates and exploit_link to 'NONE'.
    
    Parameters:
    result_df (DataFrame): DataFrame with contracts to be analyzed
    exploits_df (DataFrame): DataFrame containing information about available exploits
    
    Returns:
    DataFrame: Updated DataFrame with NONE values for contracts without exploits
    """
    # Create a set of (category, name) tuples for contracts with exploits
    exploit_pairs = set(zip(exploits_df['category'], exploits_df['name']))
    
    # Create a mask for rows that don't have an exploit
    no_exploit_mask = ~result_df.apply(lambda row: (row['Category'], row['Original']) in exploit_pairs, axis=1)
    
    # Set 'NONE' values for contracts without exploits
    result_df.loc[no_exploit_mask, 'functional_check'] = 'non-exploitable'
    result_df.loc[no_exploit_mask, 'mitigates'] = 'non-exploitable'
    result_df.loc[no_exploit_mask, 'exploit_link'] = 'non-exploitable'
    
    # Print summary
    print(f"Total contracts without exploits: {no_exploit_mask.sum()}")
    print(f"Total contracts with exploits: {len(result_df) - no_exploit_mask.sum()}")
    
    return result_df


def read_functional_and_exploits(tools, result_df):

    """
    Read functional and exploit test results for each tool and update the functional_check and mitigates columns.
    
    Parameters:
    tools (list): List of tool names to process
    result_df (DataFrame): DataFrame with columns including Category, Tool, functional_check, mitigates, etc.
    
    Returns:
    DataFrame: Updated DataFrame with functional_check and mitigates columns filled based on test results
    """

    result_df = mark_no_exploit_contracts(result_df, exploits)
    
    for tool in tools:
        file_path = f'../results/smartbugs/{tool}/results_exploits.json'
        if os.path.exists(file_path):
            print(f"Processing results for tool: {tool}")
            with open(file_path, 'r') as file:
                result_data = json.load(file)
                
                # Create a mapping to store test results by contract file
                contract_results = {}
                
                # Process failed functional tests
                for test in result_data.get('failedSanityTests', []):
                    contract_file = test.get('contractFile')
                    test_file= test.get('file')

                    if contract_file:
                        contract_results[contract_file] = {'functional': 'failed', 'exploit': None, 'test_file': test_file}
                
                # Process passed exploit tests
                for test in result_data.get('passedResults', []):
                    contract_file = test.get('contractFile')
                    test_file= test.get('file')
                    if contract_file:
                        if contract_file not in contract_results:
                            contract_results[contract_file] = {'functional': 'passed', 'exploit': 'passed', 'test_file': test_file}
                        else:
                            contract_results[contract_file]['exploit'] = 'passed'
                
                # Process failed exploit tests
                for test in result_data.get('failedResults', []):
                    contract_file = test.get('contractFile')
                    test_file= test.get('file')
                    if contract_file:
                        if contract_file not in contract_results:
                            contract_results[contract_file] = {'functional': 'passed', 'exploit': 'failed', 'test_file': test_file}
                        else:
                            contract_results[contract_file]['exploit'] = 'failed'
                
                # Update rows where Tool matches the current tool
                tool_mask = result_df['Tool'] == tool
                
                # Only update rows that don't already have 'non-exploitable' values (skip contracts with no exploits)
                update_mask = tool_mask & (result_df['functional_check'] != 'non-exploitable')
                
                # Update the DataFrame using vectorized operations where possible
                for idx, row in result_df[update_mask].iterrows():
                    # Construct contract path to match with what's in the test results
                    if tool in ['Elysium', 'SmartShield', 'sGuardPlus']:
                        contract_path = f"{row['Category']}/{row['Original']}"
                    else:
                        contract_path = f"{row['Category']}/{row['Patch']}"
                    result = contract_results.get(contract_path, {})


                    
                    # Update mitigates based on functional and exploit results
                    functional = result.get('functional', 'unknown')
                    exploit = result.get('exploit', 'unknown')
                    
                    # Update functional_check
                    if functional == 'passed' or functional == 'failed':
                         result_df.at[idx, 'functional_check'] = functional
                    
                    # Update mitigates based on functional and exploit results
                    functional = result.get('functional', 'unknown')
                    exploit = result.get('exploit', 'unknown')
                    
                    if functional == 'passed' and exploit == 'failed':
                        result_df.at[idx, 'mitigates'] = 'yes'
                    elif exploit == 'passed':
                        result_df.at[idx, 'mitigates'] = 'no'
                    test_file= result.get('test_file', None)
                    # Update exploit link
                    exploit_link = f"https://github.com/ASSERT-KTH/sb-heists/tree/main/smartbugs-curated/0.4.x/test/{test_file}"
                    result_df.at[idx, 'exploit_link'] = exploit_link


                
                # Print summary for this tool
                print(f"\nSummary for {tool}:")
                tool_rows = result_df[result_df['Tool'] == tool]
                functional_counts = tool_rows['functional_check'].value_counts().to_dict()
                mitigates_counts = tool_rows['mitigates'].value_counts().to_dict()
                print(f"functional check counts: {functional_counts}")
                print(f"Mitigates counts: {mitigates_counts}")
        else:
            print(f"Results file not found for tool: {tool}")
    
    return result_df
                
 


Total files: 91

Files per category:
category
reentrancy                   26
unchecked_low_level_calls    20
access_control               16
arithmetic                   13
bad_randomness                4
denial_of_service             4
front_running                 3
time_manipulation             3
other                         2
Name: count, dtype: int64


In [32]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [33]:
result_df=read_diff(tools, result_df)
#result_df['Detected'].fillna(None, inplace=True)
#result_df['Fixed'].fillna(None, inplace=True)

comp_tools=[
    #'Elysium', 
    'sGuard', 
    'sGuardPlus',
    'SmartFix', 
    #'SmartShield', 
    "SolGPT" ,
    'TIPS', 
]
non_comp_tools=[
    'Elysium', 
    'SmartShield'
]
result_df=read_comp(comp_tools, result_df)
result_df['COMP'].fillna("N/A", inplace=True)

result_df=read_d_f_m(tools,result_df)

result_df=read_functional_and_exploits(tools, result_df)

Total contracts without exploits: 493
Total contracts with exploits: 833
Processing results for tool: Elysium

Summary for Elysium:
functional check counts: {'passed': 48, 'non-exploitable': 47, 'failed': 26}
Mitigates counts: {'non-exploitable': 47, 'yes': 29, 'no': 20}
Processing results for tool: sGuard

Summary for sGuard:
functional check counts: {'passed': 45, 'non-exploitable': 41, 'failed': 1}
Mitigates counts: {'non-exploitable': 41, 'yes': 30, 'no': 15}
Processing results for tool: sGuardPlus

Summary for sGuardPlus:
functional check counts: {'passed': 49, 'non-exploitable': 32}
Mitigates counts: {'yes': 44, 'non-exploitable': 32, 'no': 5}
Processing results for tool: SmartFix

Summary for SmartFix:
functional check counts: {'passed': 60, 'non-exploitable': 22, 'failed': 4}
Mitigates counts: {'yes': 48, 'non-exploitable': 22, 'no': 12}
Processing results for tool: SmartShield

Summary for SmartShield:
functional check counts: {'passed': 67, 'non-exploitable': 47, 'failed': 21

#### A few functional checks to ensure data correctly captured

In [34]:
#check for missing values, if good nothing should be printed
for tool in tools:
    query= result_df[(result_df['Tool']==tool)& (result_df['Fixed']>0)]
    if len(query.values.tolist()) == 0:
        print(tool)
        print(query)
        print('----------------------')

In [35]:
# all tools should have Fixed >0 for at least one patch
for tool in tools:
    print(tool)
    assert(len(result_df[(result_df['Tool']==tool)& (result_df['Fixed']>0)].values.tolist()))>0

Elysium
sGuard
sGuardPlus
SmartFix
SmartShield
SolGPT
TIPS


In [36]:
#sanity of results checks
test_n_patches(result_df)
test_dataset(result_df)
test_diff_patches(result_df)
test_compilable_patches(result_df)
#test_valid_patches(result_df)

In [37]:
# Turn to csv
#[TODO] Uncomment to save the results
#result_df.to_csv(os.path.join(storage_path,'all_patches_stats.csv'), index=True)

### 2) Code to generate diff files

In [38]:
import difflib

def read_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.readlines()

def diff_files(file1, file2, file1_path, file2_path):
    formatted1 = read_file(file1)
    formatted2 = read_file(file2)

    diff = difflib.unified_diff(
        formatted1, formatted2,
        fromfile=file1_path, tofile=file2_path,
        lineterm=''
    )

    return '\n'.join(diff)

def patch_path(path):
    github_prefix = "https://github.com/ASSERT-KTH/RepairComp/blob/main/results/"

    dir_prefix = "../results/"

    path = path.replace(github_prefix, dir_prefix)
    return path

def og_path(path):
    github_prefix = "https://github.com/smartbugs/smartbugs-curated/tree/main/"
    dir_prefix = "../../smartbugs-curated/"
    path = path.replace(github_prefix, dir_prefix)
    return path

def generate_diffs():
    for vuln in vulnerabilities:
        # read the patches
        patches_vuln = pd.read_csv(os.path.join(storage_path, vuln + '_patches.csv'))

        # iterate patch_link and original_link from the patches using zip
        for patch_link, original_link in zip(patches_vuln['patch_link'], patches_vuln['original_link']):
            # patch_path and og_path are functions that replace the prefix of the path
            patch = patch_path(patch_link)
            original = og_path(original_link)

            #  check if patch and original exist
            if not os.path.exists(patch):
                print(f"Patch file does not exist: {patch}")
                # continue
            if not os.path.exists(original):
                print(f"Original file does not exist: {original}")
                # continue

            # diff_files is a function that returns the diff between two files
            diff = diff_files(original, patch, original_link, patch_link)

            filename = patch.replace('.sol', '.diff')
            print("Filename: ", filename)

            # Save the diff to a file
            with open(filename, 'w') as f:
                f.write(diff)

In [39]:
patches_path = os.path.join(storage_path, 'all_patches_stats.csv')
def generate_diffs_for_all(force_regenerate=True):
    """
    Generate diff files for all source code patches in all_patches_stats.csv 
    Places diff files in the same directory as each patch file.
    Uses the existing patch_path, og_path, and diff_files functions.
    
    Args:
        force_regenerate (bool): If True, regenerate even if diff file exists
    """
    all_patches_stats = pd.read_csv(patches_path)


    print(f"Processing {len(all_patches_stats)} patches for diff generation...")
    if force_regenerate:
        print("⚠️  Force regeneration enabled - will replace existing diff files")
    
    stats = {
        'total': len(all_patches_stats),
        'generated': 0,
        'replaced': 0,
        'skipped_existing': 0,
        'skipped_missing_files': 0,
        'errors': 0
    }
    
    # Iterate through all patches in all_patches_stats
    for idx, row in all_patches_stats.iterrows():
        patch_link = row['patch_link']
        original_link = row['original_link']
        
        try:
            # Convert GitHub URLs to local paths using existing functions
            patch = patch_path(patch_link)
            original = og_path(original_link)
            
            # Skip bytecode files (.bin, .hex) since we don't have source code to compare
            patch_basename = os.path.basename(patch)
            patch_ext = os.path.splitext(patch_basename)[1].lower()
            
            if patch_ext in ['.bin', '.hex']:
                print(f"Skipping bytecode file: {patch_basename}")
                stats['skipped_missing_files'] += 1
                continue
            
            # Create diff filename in the same directory as the patch file
            patch_dir = os.path.dirname(patch)
            
            # Remove extension and add .diff
            diff_basename = os.path.splitext(patch_basename)[0] + '.diff'
            diff_filename = os.path.join(patch_dir, diff_basename)
            
            # Check if diff already exists
            if os.path.exists(diff_filename):
                if force_regenerate:
                    print(f"Replacing existing diff: {diff_basename}")
                    stats['replaced'] += 1
                else:
                    print(f"Diff already exists, skipping: {diff_filename}")
                    stats['skipped_existing'] += 1
                    continue
            
            # Check if source files exist
            missing_files = []
            if not os.path.exists(patch):
                missing_files.append(f"Patch: {patch}")
            if not os.path.exists(original):
                missing_files.append(f"Original: {original}")
            
            if missing_files:
                print(f"Missing files for row {idx}:")
                for missing in missing_files:
                    print(f"  - {missing}")
                stats['skipped_missing_files'] += 1
                continue
            
            # Generate the diff using existing function (fix parameter order)
            print(f"Generating diff for: {patch_basename} -> {diff_basename}")
            diff_content = diff_files(original, patch, original_link, patch_link)
            
            # Ensure patch directory exists (should already exist if patch file exists)
            if not os.path.exists(patch_dir):
                os.makedirs(patch_dir, exist_ok=True)
            
            # Save the diff to file in the same directory as the patch
            with open(diff_filename, 'w', encoding='utf-8') as f:
                f.write(diff_content)
            
            if not os.path.exists(diff_filename) or stats['replaced'] == 0:
                stats['generated'] += 1
            
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            print(f"  Patch link: {patch_link}")
            print(f"  Original link: {original_link}")
            stats['errors'] += 1
            continue
    
    # Print summary statistics
    print(f"\n=== Diff Generation Summary ===")
    print(f"Total patches processed: {stats['total']}")
    print(f"New diffs generated: {stats['generated']}")
    print(f"Existing diffs replaced: {stats['replaced']}")
    print(f"Already existing (skipped): {stats['skipped_existing']}")
    print(f"Missing source files (skipped): {stats['skipped_missing_files']}")
    print(f"Errors encountered: {stats['errors']}")
    
    total_created = stats['generated'] + stats['replaced']
    if total_created > 0:
        print(f"\n✅ Successfully created/replaced {total_created} diff files!")
    else:
        print(f"\n⚠️  No diff files were generated or replaced.")




In [40]:
# Generate all diffs, replacing existing ones (recommended for your situation)
# If you want to avoid replacing existing diffs, set force_regenerate=False
# [TODO] Uncomment the line below to run the diff generation
#generate_diffs_for_all(force_regenerate=True)